# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [2]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

This non-commercial license of GraphLab Create is assigned to ty.walker@gmail.com and will expire on November 24, 2016. For commercial licensing options, visit https://turi.com/buy/.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.0.1 started. Logging: C:/Users/Ty/AppData/Local/Temp\graphlab_server_1468719875.log.0


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [4]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [5]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]

    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]

    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [6]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [7]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [8]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


To normalize, apply element-wise division:

In [9]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [10]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    return feature_matrix / norms, norms

To test the function, run the following:

In [11]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [12]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [13]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [14]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [15]:
prediction = predict_output(simple_feature_matrix, weights)

In [16]:
prediction

array([ 0.02675867,  0.04339256,  0.01990703, ...,  0.02289873,
        0.03178473,  0.02289873])

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [17]:
len(weights)

3

In [18]:
np.zeros(3)

array([ 0.,  0.,  0.])

In [19]:
# Get num columns in array
n = len(weights)
ro = np.zeros((n, 1))

for i in range(len(weights)):
    ro[i] = np.sum(simple_feature_matrix[:,i] * (output - prediction + weights[i] * simple_feature_matrix[:, i]))

***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

**Answer**: Between 1.62E8 and 1.75E8, which is 1.64 and 1.73 on the quiz. See P&P notebook for maths

In [20]:
ro

array([[ 79400300.03492916],
       [ 87939470.77299108],
       [ 80966698.67596565]])

In [21]:
print "{:.2E}".format(float(ro[1]))
print "{:.2E}".format(float(ro[2]))

8.79E+07
8.10E+07


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

**Answer**: Anything less than 1.62, which is 1.4 on the quiz

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [22]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    
    ro_i = np.sum(feature_matrix[:,i] * (output - prediction + weights[i] * feature_matrix[:, i]))

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
        
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
        
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
        
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [23]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [24]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, weights, l1_penalty, tolerance):
    weight_diff = []
    
    for i in range(len(weights)):
        old_weight_i = weights[i]
        weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
        weight_diff.append(abs(old_weight_i - weights[i]))
        
    if max(weight_diff) < tolerance:
        return weights
    else: 
        return lasso_cyclical_coordinate_descent(feature_matrix, output, weights, l1_penalty, tolerance)

Using the following parameters, learn the weights on the sales dataset. 

In [25]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

In [26]:
len(simple_features)

2

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`

In [27]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [28]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

In [29]:
weights

array([ 21624998.36636292,  63157246.78545421,         0.        ])

In [30]:
def get_rss(feature_matrix, weights, output):
    predictions = predict_output(feature_matrix, weights)
    residuals = predictions - output
    RSS = sum(map(lambda x: x**2, residuals))
    return RSS

In [31]:
rss = get_rss(normalized_simple_feature_matrix, weights, output)

In [32]:
"{:.2E}".format(rss)

'1.63E+15'

***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset?
2. Which features had weight zero at convergence?

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [33]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [34]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [35]:
(feature_matrix_train, output_train) = get_numpy_data(train_data, all_features, 'price')
(feature_matrix_test, output_test) = get_numpy_data(test_data, all_features, 'price')

(normalized_feature_matrix_train, train_norms) = normalize_features(feature_matrix_train)
(normalized_feature_matrix_test, test_norms) = normalize_features(feature_matrix_test)

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [36]:
initial_weights = np.zeros(len(all_features) + 1)
l1_penalty = 1e7
tolerance = 1

weights1e7 = lasso_cyclical_coordinate_descent(normalized_feature_matrix_train, output_train, initial_weights, l1_penalty, tolerance)

In [37]:
range(1, len(all_features))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [38]:
weights1e7

array([ 24429600.60933314,         0.        ,         0.        ,
        48389174.35227978,         0.        ,         0.        ,
         3317511.16271982,   7329961.9848964 ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ])

In [39]:
def get_nnz(weights, features):
    # Will return the names of features with non-zero weights
    nnz = {}
    nnz['intercept'] = weights[0]
    
    for i in range(1, len(features) + 1):
        if weights[i] != 0:
            nnz[features[i-1]] = weights[i]
    
    return nnz

In [40]:
get_nnz(weights1e7, all_features)

{'intercept': 24429600.609333135,
 'sqft_living': 48389174.352279782,
 'view': 7329961.9848963991,
 'waterfront': 3317511.162719816}

***QUIZ QUESTION***

What features had non-zero weight in this case?

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [41]:
l1_penalty = 1e8
weights1e8 = lasso_cyclical_coordinate_descent(normalized_feature_matrix_train, output_train, initial_weights, l1_penalty, tolerance)

In [42]:
weights1e8

array([ 71114625.75280938,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ])

In [43]:
get_nnz(weights1e8, all_features)

{'intercept': 71114625.752809376}

***QUIZ QUESTION***

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [46]:
tolerance = 5e5
l1_penalty = 1e4

weights1e4 = lasso_cyclical_coordinate_descent(normalized_feature_matrix_train, output_train, initial_weights, l1_penalty, tolerance)

In [45]:
weights1e4

array([ 77779073.91265225, -22884012.25023361,  15348487.08089996,
        92166869.69883074,  -2139328.0824278 ,  -8818455.54409492,
         6494209.73310655,   7065162.05053198,   4119079.21006765,
        18436483.52618776, -14566678.54514342,  -5528348.75179426,
       -83591746.20730537,   2784276.46012858])

In [49]:
get_nnz(weights1e4, all_features)

{'bathrooms': 15213940.268244578,
 'bedrooms': -22884295.07136539,
 'condition': 4233961.5091190152,
 'floors': -8762739.4401598126,
 'grade': 18775768.110632945,
 'intercept': 77857584.281944066,
 'sqft_above': -15014105.221118076,
 'sqft_basement': -5636245.6297332272,
 'sqft_living': 92657355.136650041,
 'sqft_lot': -2138355.8326287912,
 'view': 7059098.2212281525,
 'waterfront': 6494741.5469957506,
 'yr_built': -84025991.318831652,
 'yr_renovated': 2786618.2734857132}

***QUIZ QUESTION***

What features had non-zero weight in this case? All of them

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [61]:
def normalize_weights(feature_matrix, output, l1_penalty, tolerance):
    normalized_features, norms = normalize_features(feature_matrix)
    
    initial_weights = np.zeros(feature_matrix.shape[1])   
    weights = lasso_cyclical_coordinate_descent(normalized_features, output, initial_weights, l1_penalty, tolerance)
    weights_normalized = weights / norms
    
    return weights_normalized

In [62]:
normalize_weights(feature_matrix_train, output_train, 1e7, 1.)

array([  1.85285533e+05,   0.00000000e+00,   0.00000000e+00,
         1.61317456e+02,   0.00000000e+00,   0.00000000e+00,
         2.87664700e+05,   6.91937057e+04,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00])

In [60]:
feature_matrix_train.shape[1]

14L

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [63]:
feature_matrix_test, output_test

(array([[  1.00000000e+00,   3.00000000e+00,   1.00000000e+00, ...,
           0.00000000e+00,   1.92700000e+03,   0.00000000e+00],
        [  1.00000000e+00,   4.00000000e+00,   3.00000000e+00, ...,
           9.70000000e+02,   1.97900000e+03,   0.00000000e+00],
        [  1.00000000e+00,   3.00000000e+00,   2.00000000e+00, ...,
           0.00000000e+00,   1.94100000e+03,   0.00000000e+00],
        ..., 
        [  1.00000000e+00,   4.00000000e+00,   2.50000000e+00, ...,
           0.00000000e+00,   2.01400000e+03,   0.00000000e+00],
        [  1.00000000e+00,   4.00000000e+00,   2.50000000e+00, ...,
           0.00000000e+00,   2.01400000e+03,   0.00000000e+00],
        [  1.00000000e+00,   2.00000000e+00,   7.50000000e-01, ...,
           0.00000000e+00,   2.00900000e+03,   0.00000000e+00]]),
 array([ 310000.,  650000.,  233000., ...,  610685.,  400000.,  402101.]))

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [66]:
# l1_penalty = 1e7, tolerance = 1
normalized_weights1e7 = normalize_weights(feature_matrix_train, output_train, 1e7, 1)
normalized_weights1e8 = normalize_weights(feature_matrix_train, output_train, 1e8, 1)
normalized_weights1e4 = normalize_weights(feature_matrix_train, output_train, 1e4, 5e5)

In [67]:
rss1e7 = get_rss(feature_matrix_test, normalized_weights1e7, output_test)
rss1e8 = get_rss(feature_matrix_test, normalized_weights1e8, output_test)
rss1e4 = get_rss(feature_matrix_test, normalized_weights1e4, output_test)

In [68]:
print "RSS for 1e7: {:.2E}".format(rss1e7)
print "RSS for 1e8: {:.2E}".format(rss1e8)
print "RSS for 1e4: {:.2E}".format(rss1e4)

RSS for 1e7: 2.76E+14
RSS for 1e8: 5.37E+14
RSS for 1e4: 2.28E+14


***QUIZ QUESTION***

Which model performed best on the test data?